In [1]:
%load_ext autoreload
%autoreload 2
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score
from ml_utils.metrics import aur_pr
from answer_judge import load_judge

In [2]:
data_train=ReadData('/root/autodl-fs/hallscope/ScienceQA/train').read_all()
#data_val=ReadData('/root/autodl-fs/hallscope/ScienceQA/val').read_all()
#data_test=ReadData('/root/autodl-fs/hallscope/ScienceQA/test').read_all()

In [3]:
df_train=pd.DataFrame(data_train)
#df_val=pd.DataFrame(data_val)
#df_test=pd.DataFrame(data_test)

In [4]:
df_train.head()

,question,answer,most_likely
0,[Question:Which of these states is farthest no...,[tensor(0)],{'embedding': [[[ 0.00029945 -0.00512695 -0.00...
1,[Question:Identify the question that Tom and J...,[tensor(1)],{'embedding': [[[ 0.02087402 -0.02612305 0.00...
2,[Question:Identify the question that Kathleen ...,[tensor(0)],{'embedding': [[[ 0.02148438 -0.0088501 0.00...
3,[Question:Which tense does the sentence use?\n...,[tensor(1)],{'embedding': [[[ 0.00448608 -0.02160645 0.00...
4,[Question:Complete the sentence.\nSewing an ap...,[tensor(1)],{'embedding': [[[ 0.00389099 0.00149536 0.00...


In [5]:
i=0
df_train.iloc[i]['most_likely']['response']

['D.Oklahoma']

In [6]:
df_train.iloc[i]['question']

['Question:Which of these states is farthest north?\nAnswer the Question with following choices.\nChoices: A.West Virginia B.Louisiana C.Arizona D.Oklahoma\n']

In [7]:
embedding_train=df_train['most_likely'].apply(lambda x: x['embedding']).to_list()
embedding_train=np.concatenate(embedding_train,axis=0)
embedding_train.shape

(2687, 33, 4096)

In [8]:
embedding_val=df_val['most_likely'].apply(lambda x: x['embedding']).to_list()
embedding_val=np.concatenate(embedding_val,axis=0)
embedding_val.shape

NameError: name 'df_val' is not defined

In [9]:
embedding_test=df_test['most_likely'].apply(lambda x: x['embedding']).to_list()
embedding_test=np.concatenate(embedding_test,axis=0)
embedding_test.shape

(520, 33, 4096)

In [9]:
#gt_label=df_train['answer'].to_list()+df_val['answer'].to_list()+df_test['answer'].to_list()
gt_label=df_train['answer'].to_list()
gt_label=np.concatenate(gt_label,axis=0)
gt_label.shape

(2687,)

In [10]:
gt_label[0]

np.int64(0)

In [11]:
judge=load_judge('ScienceQA')

In [12]:
#responses=df_train['most_likely'].apply(lambda x: x['response']).to_list()+df_val['most_likely'].apply(lambda x: x['response']).to_list()+df_test['most_likely'].apply(lambda x: x['response']).to_list()

responses=df_train['most_likely'].apply(lambda x: x['response']).to_list()

rsps=[]
for x in responses:
    rsps+=x
    
is_hall=[]
for i,x in enumerate(rsps):
    is_hall.append(judge.check(x,gt_label[i]))

is_hall=np.array(is_hall)
print(f'There are {len(is_hall)} responses. {sum(is_hall==1)} answer correctly and {sum(is_hall==0)} answer wrongly')

There are 2687 responses. 1452 answer correctly and 1235 answer wrongly


In [15]:
len_train = len(embedding_train)
train_rate = 0.8

embedding_train_val = embedding_train
embedding_train = embedding_train_val[:int(len_train*train_rate)]
embedding_val = embedding_train_val[int(len_train*train_rate):]

#embedding_test = embedding_test

y_train = is_hall[:int(len_train*train_rate)]
y_val = is_hall[int(len_train*train_rate):len_train]
y_test = is_hall[len_train:]


'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
print(f'trainset length:{len(y_train)}')
print(f'validset length:{len(y_val)}')
print(f'testset length:{len(y_test)}')
print(
    f'trainset postive: {sum(y_train==1)} negative: {sum(y_train==0)}')
print(
    f'valset postive: {sum(y_val==1)} negative: {sum(y_val==0)}')
#print(
#    f'testset postive: {sum(y_test==1)} negative: {sum(y_test==0)}')

trainset length:1719
validset length:430
testset length:538
trainset postive: 931 negative: 788
valset postive: 218 negative: 212


In [16]:
%load_ext autoreload 
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [17]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [18]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,y_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.5232603427384457
found best n_components:1,layer:1 Score: 0.5297191448848884
found best n_components:1,layer:4 Score: 0.5351718019733426
found best n_components:1,layer:5 Score: 0.5580859442617275


 18%|█▊        | 2/11 [00:01<00:08,  1.05it/s]

found best n_components:3,layer:5 Score: 0.5617859615717501


100%|██████████| 11/11 [00:11<00:00,  1.01s/it]


In [19]:
best_paras

{'n_components': 3, 'layer': 5, 'best_score': np.float64(0.5617859615717501)}

In [20]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [21]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(y_val)

In [22]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'train set acc: {accuracy_score(y_train,preds)} AUROC:{roc_auc_score(y_train,scores)} AUC-PR:{aur_pr(scores,y_train)}')

train set acc: 0.535194880744619 AUROC:0.5075958115011968 AUC-PR:0.5577071742922075


In [22]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(y_test,preds)} AUROC:{roc_auc_score(y_test,scores)} AUC-PR:{aur_pr(scores,y_test)}')

ValueError: Found input variables with inconsistent numbers of samples: [4761, 520]

Train Linear Prob to Get Better Results

In [18]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=y_test

In [19]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [20]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores = torch.sigmoid(output).cpu().data.numpy()
preds=scores>0.5
print(f'test set acc: {accuracy_score(y_test,preds)} AUROC:{roc_auc_score(y_test,scores)} AUC-PR:{aur_pr(scores,y_test)} f1:{f1_score(y_test,preds)}')

test set acc: 0.925 AUROC:0.9499296507936509 AUC-PR:0.9920313052733323 f1:0.9575711861210635
